In [18]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

import os
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
matplotlib.style.use('ggplot')
%matplotlib inline

In [2]:
!ls input/dataset

chain_info.csv hotel_info.csv test_set.csv   train_set.csv


## EDA

In [3]:
train_df = pd.read_csv('input/dataset/train_set.csv', names=['image_id', 'hotel_id', 'image_url', 'image_source', 'upload_timestamp'])

In [4]:
train_df.head()

,image_id,hotel_id,image_url,image_source,upload_timestamp
0,3485,18187,https://traffickcam.com/images/2016/10/2015090...,traffickcam,9/9/15 17:23
1,3486,18187,https://traffickcam.com/images/2016/10/2015090...,traffickcam,9/9/15 17:23
2,3663,73224,https://traffickcam.com/images/2016/10/2015091...,traffickcam,9/17/15 19:33
3,2586939,86350,https://traffickcam.com/images/2017/2/20160125...,traffickcam,1/25/16 19:12
4,2586950,1533,https://traffickcam.com/images/2017/2/20160125...,traffickcam,1/25/16 17:23


In [14]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1124215 entries, 0 to 1124214
Data columns (total 5 columns):
 #   Column            Non-Null Count    Dtype 
---  ------            --------------    ----- 
 0   image_id          1124215 non-null  int64 
 1   hotel_id          1124215 non-null  int64 
 2   image_url         1124215 non-null  object
 3   image_source      1124215 non-null  object
 4   upload_timestamp  1124215 non-null  object
dtypes: int64(2), object(3)
memory usage: 42.9+ MB


In [12]:
train_df[train_df['hotel_id']==37821]

,image_id,hotel_id,image_url,image_source,upload_timestamp
8381,2631490,37821,https://traffickcam.com/images/2017/8/20160731...,traffickcam,7/31/16 3:58
8382,2631492,37821,https://traffickcam.com/images/2017/8/20160731...,traffickcam,7/31/16 3:58
668497,6324595,37821,https://i.travelapi.com/hotels/1000000/450000/...,travel_website,2019-12-19 02:30:24
668498,6324596,37821,https://i.travelapi.com/hotels/1000000/450000/...,travel_website,2019-12-19 02:30:24
668499,6324597,37821,https://i.travelapi.com/hotels/1000000/450000/...,travel_website,2019-12-19 02:30:24
...,...,...,...,...,...
668564,6324663,37821,https://i.travelapi.com/hotels/1000000/450000/...,travel_website,2019-12-19 02:30:24
668565,6324664,37821,https://i.travelapi.com/hotels/1000000/450000/...,travel_website,2019-12-19 02:30:24
668566,6324665,37821,https://i.travelapi.com/hotels/1000000/450000/...,travel_website,2019-12-19 02:30:24
668567,6324666,37821,https://i.travelapi.com/hotels/1000000/450000/...,travel_website,2019-12-19 02:30:24


In [8]:
hotel_df = pd.read_csv('input/dataset/hotel_info.csv')

In [9]:
hotel_df.head()

,hotel_id,hotel_name,chain_id,latitude,longitude
0,391,Extended Stay America - Fairbanks - Old Airpor...,72,64.83538,-147.82330
1,392,Hilton Hangzhou Qiandao Lake Resort,3,29.60819,119.07290
2,393,Taj Lands End,-1,19.04391,72.81879
3,395,Cambridge Suites Hotel Sydney,-1,46.13663,-60.19551
4,396,Tamanu Beach,14,-18.84213,-159.78794


In [15]:
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   hotel_id    50000 non-null  int64  
 1   hotel_name  50000 non-null  object 
 2   chain_id    50000 non-null  int64  
 3   latitude    50000 non-null  float64
 4   longitude   50000 non-null  float64
dtypes: float64(2), int64(2), object(1)
memory usage: 1.9+ MB


In [10]:
hotel_df[hotel_df['hotel_id']==18934]

,hotel_id,hotel_name,chain_id,latitude,longitude
13735,18934,HYATT house Raleigh Durham Airport,1,35.85567,-78.84269


In [119]:
chain_df = pd.read_csv('input/dataset/chain_info.csv')

In [175]:
chain_df.head(80)

,chain_id,chain_name
0,-1,unknown
1,0,Best Western
2,1,Hyatt
3,2,Marriott
4,3,Hilton
...,...,...
75,75,Motel 6
76,76,Sheraton
77,77,Homewood Suites
78,78,SpringHill Suites


In [16]:
chain_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   chain_id    93 non-null     int64 
 1   chain_name  93 non-null     object
dtypes: int64(1), object(1)
memory usage: 1.6+ KB


## Local Mini Model
Testing to see if I am able to run a model with a subset of the Train data. The train2 and test2 datasets are extracted from the original Train dataset from chains 1, 22, and 39.

In [21]:
img_width = 640
img_height = 360
batch_size =10

# this is the augmentation configuration we will use for training
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1. / 255)

# this is the augmentation configuration we will use for testing:
# only rescaling (we do not want to modify the testing data)
test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1. / 255)


# The generator object. 
train_generator = train_datagen.flow_from_directory(
    './images/train2',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    './images/test2',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 155 images belonging to 3 classes.
Found 70 images belonging to 3 classes.


In [33]:
img_width, img_height = 640, 360

nb_train_samples = 155
nb_validation_samples = 70
epochs = 5
batch_size = 50

In [34]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(640, 360, 3)))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(3,
                activation='softmax',
                kernel_regularizer=keras.regularizers.l2(0.01)))

In [35]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [36]:
# m = model.fit_generator(
#         train_generator,
#         steps_per_epoch=nb_train_samples // batch_size,
#         epochs=epochs,
#         validation_data=validation_generator,
#         validation_steps=nb_validation_samples // batch_size)

Epoch 1/5
3/3 [==============================] - 56s 20s/step - loss: 2.2423 - accuracy: 0.3682 - val_loss: 3.4295 - val_accuracy: 0.2000
Epoch 2/5
3/3 [==============================] - 51s 13s/step - loss: 2.0194 - accuracy: 0.4750 - val_loss: 1.0024 - val_accuracy: 0.6462
Epoch 3/5
3/3 [==============================] - 46s 13s/step - loss: 0.9922 - accuracy: 0.5267 - val_loss: 1.3985 - val_accuracy: 0.2154
Epoch 4/5
3/3 [==============================] - 51s 11s/step - loss: 1.2320 - accuracy: 0.3953 - val_loss: 1.0755 - val_accuracy: 0.5385
Epoch 5/5
3/3 [==============================] - 47s 12s/step - loss: 1.0490 - accuracy: 0.4939 - val_loss: 0.9887 - val_accuracy: 0.6615


## Splitting 1M Train Dataset into Train and Test
Original Test dataset of 17k images have occlusions that the original Train dataset does not. To make things easier in the preliminary stages of this project, we will only train and test on the unoccluded Train dataset.

In [ ]:
#for folder in train
    # loop through each chain
    # loop through each location in the chain folder
        #create test chain folder for each train chain folder
        # if traffic cam folder exists:
        # loop through trafficcam folder
            #count length of folder 
            # take random 20 percent of images
            # (take 20 (.2 * len of folder) random numbers from 
            #1-100 (len of folder) out of 100 images)
            #NOTE: also grab a list of the image names
        # if travel folder exists:
        # loop through travel folder
            
# creating a fxn 

#count length of folder 
            # take random 20 percent of images
            # (take 20 (.2 * len of folder) random numbers from 
            #1-100 (len of folder) out of 100 images)
            #NOTE: also grab a list of the image names

In [102]:
import random

directory = './images/train/'
chains = []
for chain in range(-1,93):
    chains.append(directory+str(chain))
chains
def hotel_split(chains):
    
    for chain in chains:
        locations = [chain+'/'+f for f in os.listdir(chain)]
        for location in locations:
            if 'travel_website' in os.listdir(location):
                travelwebsite = location+'/'+'travel_website'
                # now we want to grab 20% of the images from travel website
                images = [travelwebsite+'/'+f for f in os.listdir(travelwebsite)]
                for image in images:
                    #generate a rand num so 20% of the time it will satisfy a condition
                    x = random.rand()
                    if x <= .2:
                        # then copy image to another folder test 20%
                        
                        # else copy image to another folder train 80%

hotel_images(chains)

['1069', '1855', '4729', '2886', '23278', '36598', '11188', '2288', '8574', '5865', '10892', '5059', '39372', '26994', '2618', '763', '8110', '6617', '3365', '23849', '10859', '37647', '21722', '4140', '89643', '4372', '6425', '3157', '116569', '4716', '6041', '9009', '35574', '98355', '3701', '10401', '4524', '86861', '11715', '32597', '22705', '7167', '35580', '2627', '11527', '2243', '27421', '10091', '75406', '11143', '5066', '46278', '8117', '39375', '43538', '113681', '85270', '11349', '5896', '10861', '106195', '9667', '3953', '2881', '204289', '8587', '38405', '203890', '6845', '7997', '127216', '206986', '32769', '7399', '790', '33026', '5253', '4381', '34203', '2076', '78910', '7736', '2244', '10096', '14436', '7352', '2620', '28836', '1899', '5405', '11712', '25527', '569', '8128', '3706', '12351', '1051', '4523', '1263', '4711', '9832', '27284', '61491', '111921', '121788', '1435', '4147', '5068', '52824', '44941', '2843', '19576', '7955', '8121', '560', '6887', '6289', '18

In [117]:
import shutil
images = []
for _,_,files in os.walk("./images/train/1"):
    for name in files:
        images.append(os.path.join("./images/train/1", name))
images
#shutil.copy(source file from images list, destination file) file from images
shutil.copy(images[2], './images/'+images[2].split('/')[-1])

FileNotFoundError: [Errno 2] No such file or directory: '/Users/lauranguyen/Desktop/Galvanize/capstones/Hotels-50K/images/train/1/3928997.jpg'

In [114]:
'./images/'+images[1].split('/')[-1]

'./images/3897091.jpg'

In [116]:
find . -type f -print0 | xargs -0 mv -t .

/Users/lauranguyen/Desktop/Galvanize/capstones/Hotels-50K


In [176]:
# for chain in range(-1,93):
#         print(directory+str(chain))

In [50]:
!ls ./images/train/4

10238 1708  1805  33978 38431 694   7149  7623


## 1M Dataset

In [125]:
img_width = 640
img_height = 360
batch_size =10

# this is the augmentation configuration we will use for training
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1. / 255,
                validation_split=0.2)


# The generator object. 
train_generator = train_datagen.flow_from_directory(
    './images/train',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    './images/train',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')

Found 171088 images belonging to 90 classes.
Found 42729 images belonging to 90 classes.


In [129]:
img_width, img_height = 640, 360

nb_train_samples = 171088
nb_validation_samples = 42729
epochs = 5
batch_size = 50

In [133]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(640, 360, 3)))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(90,
                activation='softmax',
                kernel_regularizer=keras.regularizers.l2(0.01)))

In [135]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [136]:
# mod = model.fit_generator(
#         train_generator,
#         steps_per_epoch=nb_train_samples // batch_size,
#         epochs=epochs,
#         validation_data=validation_generator,
#         validation_steps=nb_validation_samples // batch_size)

# DO NOT RE-RUN -- TAKES 5 HOURS!

Epoch 1/5
3421/3421 [==============================] - 3680s 1s/step - loss: 3.0307 - accuracy: 0.5095 - val_loss: 2.5701 - val_accuracy: 0.5245
Epoch 2/5
3421/3421 [==============================] - 3706s 1s/step - loss: 2.5884 - accuracy: 0.5244 - val_loss: 2.6558 - val_accuracy: 0.4883
Epoch 3/5
3421/3421 [==============================] - 3609s 1s/step - loss: 2.5186 - accuracy: 0.5258 - val_loss: 2.7740 - val_accuracy: 0.4769
Epoch 4/5
3421/3421 [==============================] - 3642s 1s/step - loss: 2.4649 - accuracy: 0.5316 - val_loss: 3.7027 - val_accuracy: 0.4334
Epoch 5/5
3421/3421 [==============================] - 4114s 1s/step - loss: 2.4502 - accuracy: 0.5280 - val_loss: 3.1841 - val_accuracy: 0.4399


### Without 'Unknown' Hotel Chains

In [150]:
img_width = 640
img_height = 360
batch_size =50

# this is the augmentation configuration we will use for training
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1. / 255,
                validation_split=0.2)


# The generator object. 
train_generator = train_datagen.flow_from_directory(
    './images/train',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    './images/train',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')

Found 80817 images belonging to 89 classes.
Found 20162 images belonging to 89 classes.


In [151]:
img_width, img_height = 640, 360

nb_train_samples = 80817
nb_validation_samples = 20162
epochs = 5
batch_size = 50

In [155]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(640, 360, 3)))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(89,
                activation='softmax',
                kernel_regularizer=keras.regularizers.l2(0.01)))

In [156]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [157]:
mod_no_unk = model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size)

Epoch 1/5
   8/1616 [..............................] - ETA: 5:28:56 - loss: 10.0712 - accuracy: 0.0112

KeyboardInterrupt: 

## Top Chains Data

In [177]:
#0 best western, 75 motel 6, 2 marriott, 3 hilton, 1 hyatt, 86 la quinta, 57 travellodge, 
#47 kimpton, 64 candelwood suites, 60 wyndham, 71 crowne plaza, 72 extended stay america, 
# 73, ramada, 74 embassy suites, 70 econo lodge, 67 red roof

In [192]:
img_width = 640
img_height = 360
batch_size = 10

# this is the augmentation configuration we will use for training
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1. / 255,
                validation_split=0.2)


# The generator object. 
train_generator = train_datagen.flow_from_directory(
    './images/top_chains',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    './images/top_chains',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')

Found 14085 images belonging to 6 classes.
Found 3520 images belonging to 6 classes.


In [271]:
img_width, img_height = 640, 360

nb_train_samples = 14085
nb_validation_samples = 3520
epochs = 5
batch_size = 15

In [275]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(640, 360, 3)))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

# model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(6,
                activation='softmax',
                kernel_regularizer=keras.regularizers.l2(0.0001)))

In [276]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [200]:
mod_top_chains = model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size)

Epoch 1/5
939/939 [==============================] - 1113s 1s/step - loss: 2.8114 - accuracy: 0.2352 - val_loss: 1.8267 - val_accuracy: 0.2278
Epoch 2/5
939/939 [==============================] - 1175s 1s/step - loss: 1.6645 - accuracy: 0.3004 - val_loss: 2.2616 - val_accuracy: 0.2504
Epoch 3/5
939/939 [==============================] - 1045s 1s/step - loss: 1.5110 - accuracy: 0.3974 - val_loss: 2.3916 - val_accuracy: 0.2376
Epoch 4/5
939/939 [==============================] - 1121s 1s/step - loss: 1.2702 - accuracy: 0.5233 - val_loss: 3.2426 - val_accuracy: 0.2312
Epoch 5/5
939/939 [==============================] - 1359s 1s/step - loss: 1.0315 - accuracy: 0.6211 - val_loss: 3.9498 - val_accuracy: 0.2286


In [277]:
model.summary()

Model: "sequential_41"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_98 (Conv2D)           (None, 638, 358, 32)      896       
_________________________________________________________________
max_pooling2d_97 (MaxPooling (None, 319, 179, 32)      0         
_________________________________________________________________
conv2d_99 (Conv2D)           (None, 317, 177, 32)      9248      
_________________________________________________________________
max_pooling2d_98 (MaxPooling (None, 158, 88, 32)       0         
_________________________________________________________________
flatten_39 (Flatten)         (None, 444928)            0         
_________________________________________________________________
dense_78 (Dense)             (None, 64)                28475456  
_________________________________________________________________
dropout_39 (Dropout)         (None, 64)              

### Hyperparameter Tuning
#### 1.

In [285]:
img_width, img_height = 640, 360

nb_train_samples = 14085
nb_validation_samples = 3520
epochs = 5
batch_size = 15

In [286]:
mod2 = keras.models.Sequential()
mod2.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(640, 360, 3)))
mod2.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

# model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
# model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

mod2.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
mod2.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

mod2.add(keras.layers.Flatten())
mod2.add(keras.layers.Dense(64, activation='relu'))
mod2.add(keras.layers.Dropout(0.5))
mod2.add(keras.layers.Dense(6,
                activation='softmax',
                kernel_regularizer=keras.regularizers.l2(0.0001)))

In [287]:
mod2.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [288]:
mod_top_chains2 = mod2.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size)

Epoch 1/5
939/939 [==============================] - 2674s 3s/step - loss: 3.0703 - accuracy: 0.2197 - val_loss: 1.7446 - val_accuracy: 0.2410
Epoch 2/5
148/939 [===>..........................] - ETA: 59:07 - loss: 1.7432 - accuracy: 0.2437

KeyboardInterrupt: 

In [274]:
mod2.summary()

Model: "sequential_40"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_96 (Conv2D)           (None, 638, 358, 32)      896       
_________________________________________________________________
max_pooling2d_95 (MaxPooling (None, 319, 179, 32)      0         
_________________________________________________________________
conv2d_97 (Conv2D)           (None, 317, 177, 64)      18496     
_________________________________________________________________
max_pooling2d_96 (MaxPooling (None, 158, 88, 64)       0         
_________________________________________________________________
flatten_38 (Flatten)         (None, 889856)            0         
_________________________________________________________________
dense_76 (Dense)             (None, 64)                56950848  
_________________________________________________________________
dropout_38 (Dropout)         (None, 64)              

#### 2. 
Great training performance, however the validation loss decreases or fluctuates wildly while the validation accuracy stays around 23 percent.

In [267]:
img_width, img_height = 640, 360

nb_train_samples = 14085
nb_validation_samples = 3520
epochs = 10
batch_size = 25

In [268]:
mod3 = keras.models.Sequential()
mod3.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(640, 360, 3)))
mod3.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

# model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
# model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

mod3.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
mod3.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

mod3.add(keras.layers.Flatten())
mod3.add(keras.layers.Dense(64, activation='relu'))
mod3.add(keras.layers.Dropout(0.6))
mod3.add(keras.layers.Dense(6,
                activation='softmax',
                kernel_regularizer=keras.regularizers.l2(0.0001)))

In [269]:
mod3.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [227]:
mod_top_chains3 = mod3.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size)

Epoch 1/10
563/563 [==============================] - 1033s 2s/step - loss: 3.9877 - accuracy: 0.2307 - val_loss: 1.7561 - val_accuracy: 0.2321
Epoch 2/10
563/563 [==============================] - 904s 2s/step - loss: 1.7594 - accuracy: 0.2272 - val_loss: 1.7382 - val_accuracy: 0.2407
Epoch 3/10
563/563 [==============================] - 1092s 2s/step - loss: 1.7383 - accuracy: 0.2442 - val_loss: 1.7463 - val_accuracy: 0.2336
Epoch 4/10
563/563 [==============================] - 1369s 2s/step - loss: 1.7048 - accuracy: 0.2554 - val_loss: 1.9695 - val_accuracy: 0.2486
Epoch 5/10
563/563 [==============================] - 3688s 7s/step - loss: 1.6424 - accuracy: 0.3156 - val_loss: 2.0356 - val_accuracy: 0.2364
Epoch 6/10
563/563 [==============================] - 2835s 5s/step - loss: 1.5937 - accuracy: 0.3489 - val_loss: 2.2763 - val_accuracy: 0.2471
Epoch 7/10
563/563 [==============================] - 2009s 4s/step - loss: 1.5423 - accuracy: 0.3749 - val_loss: 2.3454 - val_accuracy: 

In [270]:
mod3.summary()

Model: "sequential_39"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_94 (Conv2D)           (None, 638, 358, 32)      896       
_________________________________________________________________
max_pooling2d_93 (MaxPooling (None, 319, 179, 32)      0         
_________________________________________________________________
conv2d_95 (Conv2D)           (None, 317, 177, 64)      18496     
_________________________________________________________________
max_pooling2d_94 (MaxPooling (None, 158, 88, 64)       0         
_________________________________________________________________
flatten_37 (Flatten)         (None, 889856)            0         
_________________________________________________________________
dense_74 (Dense)             (None, 64)                56950848  
_________________________________________________________________
dropout_37 (Dropout)         (None, 64)              

5 epochs, with dropout at 0.6 worked well
#### 3.
Larger batch size give lower test accuracy around 24 percent and higher validation accuracy around 25 percent. Additionally, validation loss stays around 1.7 and slowly goes down with more epochs. This may be worth investing in for future iterations with more epochs.

In [235]:
img_width, img_height = 640, 360

nb_train_samples = 14085
nb_validation_samples = 3520
epochs = 10
batch_size = 100

In [264]:
mod4 = keras.models.Sequential()
mod4.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(640, 360, 3)))
mod4.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

# model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
# model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

mod4.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
mod4.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

mod4.add(keras.layers.Flatten())
mod4.add(keras.layers.Dense(64, activation='relu'))
mod4.add(keras.layers.Dropout(0.7))
mod4.add(keras.layers.Dense(6,
                activation='softmax',
                kernel_regularizer=keras.regularizers.l2(0.0001)))

In [265]:
mod4.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [238]:
mod_top_chains4 = mod4.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size)

Epoch 1/10
140/140 [==============================] - 199s 1s/step - loss: 13.3628 - accuracy: 0.2143 - val_loss: 1.7784 - val_accuracy: 0.2429
Epoch 2/10
140/140 [==============================] - 269s 2s/step - loss: 1.7819 - accuracy: 0.2289 - val_loss: 1.7637 - val_accuracy: 0.2400
Epoch 3/10
140/140 [==============================] - 241s 2s/step - loss: 1.7609 - accuracy: 0.2315 - val_loss: 1.7495 - val_accuracy: 0.2314
Epoch 4/10
140/140 [==============================] - 215s 2s/step - loss: 1.7546 - accuracy: 0.2461 - val_loss: 1.7539 - val_accuracy: 0.2429
Epoch 5/10
140/140 [==============================] - 196s 1s/step - loss: 1.7475 - accuracy: 0.2348 - val_loss: 1.7518 - val_accuracy: 0.2600
Epoch 6/10
140/140 [==============================] - 185s 1s/step - loss: 1.7565 - accuracy: 0.2112 - val_loss: 1.7473 - val_accuracy: 0.2114
Epoch 7/10
140/140 [==============================] - 185s 1s/step - loss: 1.7352 - accuracy: 0.2429 - val_loss: 1.7636 - val_accuracy: 0.228

In [266]:
mod4.summary()

Model: "sequential_38"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_92 (Conv2D)           (None, 638, 358, 32)      896       
_________________________________________________________________
max_pooling2d_91 (MaxPooling (None, 319, 179, 32)      0         
_________________________________________________________________
conv2d_93 (Conv2D)           (None, 317, 177, 64)      18496     
_________________________________________________________________
max_pooling2d_92 (MaxPooling (None, 158, 88, 64)       0         
_________________________________________________________________
flatten_36 (Flatten)         (None, 889856)            0         
_________________________________________________________________
dense_72 (Dense)             (None, 64)                56950848  
_________________________________________________________________
dropout_36 (Dropout)         (None, 64)              

#### 4.
Lower batch size and lower epoch to see quick results. The validation accuracy is as high as 28 percent with low test accuracy around 24 percent and validation loss at the high 1.7 mark.

In [256]:
img_width, img_height = 640, 360

nb_train_samples = 14085
nb_validation_samples = 3520
epochs = 5
batch_size = 70

In [257]:
mod = keras.models.Sequential()
mod.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(640, 360, 3)))
mod.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

mod.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
mod.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

mod.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
mod.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

mod.add(keras.layers.Flatten())
mod.add(keras.layers.Dense(64, activation='relu'))
mod.add(keras.layers.Dropout(0.9))
mod.add(keras.layers.Dense(6,
                activation='softmax',
                kernel_regularizer=keras.regularizers.l2(0.0001)))

mod.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [258]:
mod_top_chains5 = mod.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size)

Epoch 1/5
201/201 [==============================] - 316s 2s/step - loss: 3.2778 - accuracy: 0.2234 - val_loss: 1.7716 - val_accuracy: 0.2480
Epoch 2/5
201/201 [==============================] - 481s 2s/step - loss: 1.7645 - accuracy: 0.2670 - val_loss: 1.7741 - val_accuracy: 0.2340
Epoch 3/5
201/201 [==============================] - 475s 2s/step - loss: 1.7608 - accuracy: 0.2348 - val_loss: 1.7441 - val_accuracy: 0.2820
Epoch 4/5
201/201 [==============================] - 557s 3s/step - loss: 1.7585 - accuracy: 0.2385 - val_loss: 1.7448 - val_accuracy: 0.2600
Epoch 5/5
201/201 [==============================] - 394s 2s/step - loss: 1.7540 - accuracy: 0.2381 - val_loss: 1.7514 - val_accuracy: 0.2480


In [260]:
mod.summary()

Model: "sequential_37"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_89 (Conv2D)           (None, 638, 358, 32)      896       
_________________________________________________________________
max_pooling2d_88 (MaxPooling (None, 319, 179, 32)      0         
_________________________________________________________________
conv2d_90 (Conv2D)           (None, 317, 177, 32)      9248      
_________________________________________________________________
max_pooling2d_89 (MaxPooling (None, 158, 88, 32)       0         
_________________________________________________________________
conv2d_91 (Conv2D)           (None, 156, 86, 64)       18496     
_________________________________________________________________
max_pooling2d_90 (MaxPooling (None, 78, 43, 64)        0         
_________________________________________________________________
flatten_35 (Flatten)         (None, 214656)          